## Packages and Assets

In [17]:
import pandas as pd
import re
import string
import nltk
from sklearn.manifold import TSNE
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from gensim.models import Word2Vec

In [18]:
df_pp = pd.read_csv('../../assets/data/df_pp.csv')
df_pp

,title,tags,link,label
0,guide divulga recomendacoes carteiras titulare...,"['Alupar', 'B3', 'Banco do Brasil', 'BR Malls'...",https://www.moneytimes.com.br/guide-divulga-re...,1
1,em <NUM> inss deve perder <NUM> bi renuncia pr...,"['INSS', 'reforma da previdência']",https://www.sunoresearch.com.br/noticias/inss-...,0
2,amazon registra alta <NUM> cento lucro liquido...,"['Amazon', 'balanço de resultados', 'Jeff Bezos']",https://www.sunoresearch.com.br/noticias/amazo...,0
3,neoenergia neoe3 obras eolicas pec devem ter i...,"['energia eólica', 'energia solar', 'NEOE3', '...",https://www.sunoresearch.com.br/noticias/neoen...,0
4,xp coloca marcopolo pomo4 bradesco bbdc4 carte...,"['BBDC3', 'BBDC4', 'Bradesco', 'Marcopolo', 'p...",https://www.sunoresearch.com.br/noticias/xp-co...,0
...,...,...,...,...
17951,ibovespa encerra queda <NUM> cento <NUM> pontos,"['Ibovespa', 'Multiplan (MULT3)', 'Positivo Te...",https://www.suno.com.br/noticias/ibovespa-qued...,2
17952,samsung preve queda <NUM> cento lucro operacio...,"['lucro', 'Samsung']",https://www.sunoresearch.com.br/noticias/samsu...,0
17953,ibovespa abre alta banco inter bidi4 recuperan...,"['Banco Inter (BIDI11)', 'Banco Inter (BIDI11/...",https://www.suno.com.br/noticias/ibovespa-abre...,1
17954,de petrobras weg confira <NUM> apostas necton ...,"['Ações', 'Carteira Recomendada', 'Comprar ou ...",https://www.moneytimes.com.br/de-petrobras-a-w...,1


## Functions

## Loading Word2vec Model

## Word2Vec Model

In [19]:
hasModel = False
if not hasModel:

    tokenized_corpus = []
    for title in df_pp['title']:
        tokenized_corpus.append(title.split())
    model = Word2Vec(sentences=tokenized_corpus, vector_size=300, window=20, min_count=5,workers =4)
    model.save("../../assets/lstm_assets/word2vec.model")
else:
    print("Model Exists!")
    model = Word2Vec.load("../../assets/lstm_assets/word2vec.model")
    print(model.wv.most_similar('queda'))


In [20]:
model.wv.get_normed_vectors()

array([[ 0.01866219,  0.05307037,  0.01820612, ..., -0.00834392,
         0.0118602 , -0.05180585],
       [ 0.01141411,  0.04879714,  0.07736757, ..., -0.0519786 ,
        -0.01074746,  0.13137653],
       [ 0.03650901,  0.07113868,  0.00770308, ..., -0.04748521,
         0.06866655, -0.07277176],
       ...,
       [ 0.00768315,  0.05446802,  0.01475783, ..., -0.03472255,
         0.03132542, -0.0111517 ],
       [-0.00531091,  0.04363436, -0.0140716 , ..., -0.00841737,
         0.00205152,  0.02192492],
       [-0.00094877,  0.05480519,  0.00601936, ..., -0.03268391,
         0.0350043 , -0.00601681]], dtype=float32)

In [21]:
model.wv.index_to_key

['<NUM>',
 'petrobras',
 'cento',
 'vale',
 'diz',
 'itau',
 'acoes',
 'petr4',
 'ibovespa',
 'alta',
 'queda',
 'brasil',
 'apos',
 'minerio',
 'sobre',
 'carteira',
 'vale3',
 'ferro',
 'venda',
 'bi',
 'milhoes',
 'dividendos',
 'semana',
 'dolar',
 'lucro',
 'petroleo',
 'bilhoes',
 'pode',
 'acordo',
 'mercado',
 'acao',
 'fecha',
 'banco',
 'bolsonaro',
 'veja',
 'producao',
 'preco',
 'governo',
 'gasolina',
 'eua',
 'itub4',
 'nesta',
 'nova',
 'dia',
 'sobe',
 'anuncia',
 'vai',
 'bolsa',
 'coronavirus',
 'brumadinho',
 'abre',
 'cai',
 'compra',
 'xp',
 'precos',
 'mi',
 'novo',
 'china',
 'diesel',
 'registra',
 'deve',
 'bancos',
 'trimestre',
 'aprova',
 'presidente',
 'gas',
 'btg',
 'dalian',
 'inicia',
 'oferta',
 'empresas',
 'maior',
 'mil',
 'bradesco',
 'participacao',
 'radar',
 'investimentos',
 'recomendada',
 'combustiveis',
 'pib',
 'segundo',
 've',
 'eleva',
 'troca',
 'pontos',
 'semanal',
 'ano',
 'liquido',
 'reduz',
 'premenos',
 'br',
 'confira',
 'ceo',

In [22]:
word_embeddings = []
vocab = []

for i, word in enumerate(model.wv.index_to_key):
    vocab.append(word)
    word_embeddings.append(model.wv[word])

In [23]:
emb_df = pd.DataFrame(word_embeddings)
emb_df = emb_df.assign(nome=vocab)
emb_df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,nome
0,0.097217,0.276460,0.094841,0.293696,-0.252957,0.049378,0.374096,0.700678,0.173977,-0.360382,...,0.564749,0.473344,0.098551,0.822185,0.161984,0.121988,-0.043466,0.061783,-0.269873,<NUM>
1,0.061553,0.263148,0.417220,0.413032,-0.049093,-0.163121,0.094133,0.887846,0.406406,-0.234337,...,0.569322,0.245813,-0.094254,0.463210,0.550628,-0.002371,-0.280305,-0.057958,0.708475,petrobras
2,0.234994,0.457892,0.049582,0.173245,-0.298245,0.201781,0.902781,0.715421,0.043396,-0.450965,...,0.902517,0.415356,0.065992,0.677834,0.419463,0.039391,-0.305644,0.441980,-0.468403,cento
3,-0.266107,-0.149233,-0.064391,0.277019,0.346858,-0.440543,-0.529646,0.507746,-0.122968,-0.140814,...,0.199706,0.348219,0.486233,0.161540,0.487935,-0.169645,-0.156347,0.158021,-0.215844,vale
4,0.068856,0.186597,0.294242,0.383073,0.011033,-0.210873,0.012568,0.705220,0.141695,-0.165576,...,0.438833,0.308075,0.071879,0.287173,0.534181,-0.042400,-0.347397,0.106833,0.364050,diz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3423,0.000033,0.025067,-0.001474,0.024516,0.010285,-0.054421,0.008423,0.084723,0.006494,-0.021097,...,0.050366,0.031141,0.012841,0.045957,0.056486,0.005218,-0.013783,0.016684,-0.009749,turbulento
3424,0.001501,0.024682,0.007793,0.027441,0.007990,-0.037687,0.014584,0.074719,0.010566,-0.017158,...,0.047144,0.027493,0.009139,0.042837,0.047398,0.007757,-0.013382,0.008934,0.006677,custar
3425,0.002222,0.015752,0.004268,0.021231,0.004040,-0.029088,0.018884,0.059437,0.007905,-0.016497,...,0.040242,0.020638,0.005857,0.039200,0.029865,0.004007,-0.010041,0.009059,-0.003225,entre
3426,-0.001641,0.013480,-0.004347,0.020274,0.021471,-0.049033,-0.002546,0.060447,0.005167,-0.011767,...,0.028122,0.008449,0.009555,0.033860,0.033329,0.009106,-0.002600,0.000634,0.006773,exclui


In [24]:
tsne = TSNE(n_components=2,perplexity=40, n_iter=2500, random_state=23, learning_rate=100,metric='euclidean',init='pca')
X_tsne = tsne.fit_transform(np.array(word_embeddings))
X_tsne[1:4,:]
emb_df['x'] = X_tsne[:,0]
emb_df['y'] = X_tsne[:,1]
fig = px.scatter(emb_df,x='x',y='y',hover_data={'x':False, 'y':False,'nome':True})
fig.show()